In [1]:
import atomdnn

# 'float32' is used for reading data and train by default, one can set data_type to 'float64' here
atomdnn.data_type = 'float64'

# force and stress are evaluated by default, 
# if one only need to compute potential energy, then set compute_force to false
atomdnn.compute_force = True

# default value is for converting ev/A^3 to GPa
# note that: the predicted positive stress means tension and negative stress means compression
stress_unit_convert = 160.2176 

import numpy as np
import tensorflow as tf
import pickle
from atomdnn import data
from atomdnn import network
from atomdnn.data import Data
from atomdnn.data import *
from atomdnn.network import Network

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  4


## Load tensorflow dataset 

In [2]:
dataset = tf.data.experimental.load('/workspace/data/group_share/graphene_tfdataset')

2021-10-06 16:38:13.808938: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9673 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2021-10-06 16:38:13.809950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9673 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2021-10-06 16:38:13.810772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9673 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5
2021-10-06 16:38:13.811548: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9673 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [3]:
train_dataset, val_dataset, test_dataset = split_dataset(dataset,0.7,0.2,0.1,shuffle=True)

Traning data: 630 images
Validation data: 180 images
Test data: 90 images


## Build Network object from class Network 

### \_\_init\_\_(self, elements=None, num_fingerprints=None, arch=None,activation_function=None, data_type=None, import_dir=None)

- **elements:** element list, required

- **num_fingerprints:** number of fingerprints in data, required

- **arch:** number of layers of neural network

- **activation_function:** if not set, default is 'tanh'

- **import_dir:** read from the directory of a saved (imported) network, if used, all other parameters are disabled

In [ ]:
elements = ['C']
act_fun = 'tanh' # activation function
nfp = get_fingerprints_num(dataset) # number of fingerprints
arch = [50,50] # NN layers

model = Network(elements=elements,num_fingerprints=nfp,arch=arch,activation_function=act_fun)

## Train the model

### train(self, train_dataset, val_dataset, batch_size=None, epochs=None, loss_fn=None, optimizer=None, lr=None, loss_weights=None, shuffle=True)

- **train_dataset**: tensorflow dataset for training
    
- **val_dataset**: tensorflow dataset for validation
    
- **batch_size**: if not set, use 30

- **scaling**: 'std' or 'norm'
    
- **epochs**: if not set, use 1
    
- **opimizer**: if not set, use Adam
    
- **lr**: learning rate, if not set, use 0.01
    
- **loss_weights**: sets the weights for loss function, if not set, use {'pe':1,'force':0,'stress':0}

- **compute_all_loss**: set to True, if want to compute force or stress loss even when it is trained
    
- **shuffle**: training dataset is shuffled during training

### The training and validation loss history is stored in the dictionary, with the keys of 'pe_loss', 'force_loss', 'stress_loss' and 'total_loss':  
- **train_loss**  
- **val_loss** 



In [9]:
loss_weights = {'pe' : 0.01, 'force' : 1, 'stress': 0.1}

model.train(train_dataset, val_dataset, batch_size=50, lr=0.01, epochs=5,scaling='std', loss_weights=loss_weights, compute_all_loss=True,shuffle=True)

Forces are used for training.
Stresses are used for training.
Scaling factors are computed using training dataset.
Training dataset are standardized.
Validation dataset are standardized.
Training dataset will be shuffled during training.

===> Epoch 1/5 - 3.784s/epoch
     training_loss    - pe_loss: 7.094 - force_loss: 3.104 - stress_loss: 4.037 - total_loss: 3.579
     validation_loss  - pe_loss: 4.771 - force_loss: 2.086 - stress_loss: 3.931 - total_loss: 2.527

===> Epoch 2/5 - 4.300s/epoch
     training_loss    - pe_loss: 5.643 - force_loss: 2.172 - stress_loss: 3.850 - total_loss: 2.613
     validation_loss  - pe_loss: 3.471 - force_loss: 1.779 - stress_loss: 3.553 - total_loss: 2.169

===> Epoch 3/5 - 4.414s/epoch
     training_loss    - pe_loss: 3.577 - force_loss: 1.887 - stress_loss: 3.468 - total_loss: 2.270
     validation_loss  - pe_loss: 3.089 - force_loss: 1.583 - stress_loss: 3.497 - total_loss: 1.963

===> Epoch 4/5 - 4.294s/epoch
     training_loss    - pe_loss: 4.003

# Evaluate dataset

### evaluate(self,dataset, batch_size=None, compute_force=atomdnn.compute_force):
- **dataset**: tensorflow dataset to be evaluated
- **batch_size**: if not set, batch_size equals to the total size of dataset
- **compute**: if compute force, derivative data are needed

In [6]:
model.evaluate(test_dataset)

Evaluation loss is:
        pe_loss:       8.6793e+00
     force_loss:       3.4607e+00
    stress_loss:       3.7702e+00
     total_loss:       3.8570e+00

The prediction is:


{'pe': array([-136.5099332 , -153.36502206, -149.57517951, -137.59869475,
        -124.8382558 , -118.13533883, -153.14154835, -134.70615341,
        -149.96450679, -127.16005955, -155.38543964, -117.6680732 ,
        -154.04367329, -170.97124755, -118.43644831, -169.89147124,
        -117.78726752, -116.03680741, -146.17187856, -124.9586848 ,
        -128.23741664, -155.36981963, -152.28306911, -155.38237189,
        -152.63277298, -117.28881916, -153.41835937, -153.62306094,
        -129.51416793, -145.05603439, -150.17358081, -145.91307868,
        -149.62308402, -152.83943987, -154.57141097, -146.24003045,
        -145.76021344, -128.50909839, -144.99060896, -151.05341888,
        -153.79768343, -117.44627967, -145.52135071, -154.19088723,
        -136.7003158 , -121.71373037, -151.10673311, -153.82906805,
        -145.22175573, -146.626616  , -159.0531481 , -117.27946118,
        -148.90201732, -153.28939254, -148.0694924 , -117.04860229,
        -119.58228008, -115.17236224, -153

# Prediction

### predict(self, input_dict, compute_force=atomdnn.compute_force, training=False)

- **input_dict**: input dictionary data, which could be data outside the dataset
- **training**: set to False 
- **compute_force**: if compute force, derivative data are needed

In [7]:
# take two images from test dataset 
input_dict = get_input_dict(test_dataset.take(2))
model.predict(input_dict)

{'pe': array([-136.5099332 , -153.36502206]),
 'force': array([[[ 5.04922715e+00,  3.88846289e+00, -5.76527446e-01],
         [ 1.22667134e+00,  1.19997275e+00,  9.22135692e-01],
         [-1.58717859e+00,  8.76783145e+00, -5.60433405e-01],
         [-6.93273543e+00, -6.54945164e+00,  6.64847643e-01],
         [ 1.34822311e+01, -2.59522258e+00, -2.76919111e-01],
         [-6.82549443e+00,  5.64956968e+00,  9.93711543e-01],
         [ 6.19440585e-01, -1.22302331e+00, -1.03586510e+00],
         [ 3.34822075e+00, -6.80631220e+00,  8.68747528e-01],
         [-1.61850233e+00, -5.24070459e+00, -7.82759048e-03],
         [-2.46282058e+00, -1.06266820e+00,  6.10481562e-01],
         [-2.44539644e+00,  1.68998793e+00, -9.56095695e-01],
         [ 5.94787139e-01,  1.42905790e+00,  4.00582009e-01],
         [ 1.20248926e+00, -9.02423079e-01,  3.64103293e-01],
         [ 3.40491891e+00, -6.16421312e+00, -1.28161026e+00],
         [-1.15112616e+01,  8.72547186e+00,  6.77046087e-01],
         [ 4.13

# Save trained model

### save(obj, model_dir, descriptor=None)

- **obj**: Network object

- **model_dir**: directory for saving the trained model

- **descriptor**: descriptor parameters used to generate fingerprints, if set, a parameters file is generated for LAMMPS simulation

In [ ]:
descriptor = {'name': 'acsf', 
              'cutoff': 6.5001,
              'etaG2':[0.01,0.025,0.05,0.075,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.8,1,1.5,2,3,5,10], 
              'etaG4': [0.01], 
              'zeta': [0.08,0.1,0.15,0.2,0.3,0.35,0.5,0.6,0.8,1.,1.5,2.,3.0,4.,5.5,7.0,10.0,25.0,50.0,100.0],
              'lambda': [1.0, -1.0]}

save_dir = '/workspace/data/weigao/machine_learning/graphene.tfdnn'
network.save(model,save_dir,descriptor=descriptor)

In [ ]:
# print signature
network.print_signature(save_dir)

# Load the saved model for continuous training and prediction

**load(model_dir)**

- **model_dir**: saved model directory

In [ ]:
save_dir = '/workspace/data/weigao/machine_learning/graphene.tfdnn'
imported_model = network.load(save_dir)

In [ ]:
imported_model.train(train_dataset, val_dataset, batch_size=50, lr=0.01, epochs=5,scaling='std', loss_weights=loss_weights, shuffle=True)

In [ ]:
imported_model.evaluate(test_dataset)

In [ ]:
input_dict = get_input_dict(test_dataset.take(2))
imported_model.predict(input_dict)